In [1]:
import pennylane as qml
import numpy as np

#### Kronecker Product
###### Points to Remember
1. The state of a composite system is defined via tensor product operations
2. The system is represented using a density matrix in case of noisy systems  

Assuming we have two states $\ket{\psi}_A$ and $\ket{\psi}_B$, living in Hilbert spaces $\mathcal{H}_A$ and $\mathcal{H}_B$, respectively, the joint state of the system is expressed as $\mathcal{H}_A \otimes \mathcal{H}_B$
$$\ket{\psi}_{AB} = \ket{\psi}_A \otimes \ket{\psi}_B \in \mathcal{H}_A \otimes \mathcal{H}_B$$

Similarly if a system is comprised of two independently prepared noisy states given by the density matrices $\rho_A$ and $\rho_B$ respectively the composite state of the two systems is $$\rho_{AB} = \rho_A \otimes \rho_B$$

This operation between finite and discrete matrices is also known as the _**Kronecker Product**_

##### Solution to Codercise N.3.1
Building the composite density matrix using numpy

In [2]:
def composite_density_matrix(rho: np.array(np.array(complex)), sigma: np.array(np.array(complex))) -> np.array(np.array(complex)):
    """
    inputs
    =======
    rho: density matrix of the first input state
    sigma: density matrix of the second input state
    
    outputs
    =======
    density matrix for the composite system
    """
    return np.kron(rho, sigma)

#### Caveat
Not all composite states can be described using the above Kronecker Product. If the two states are entangled then $$\rho_{AB} \neq \rho_A \otimes \rho_B$$
In this case the composite state is definred using the partial trace operation. 
##### Explanation:
If the composite system is defined using the density matrix $\rho_{AB}$ AND we would like to describe the state of _**one**_ of the subsystems $A$, we define a partial trace over the Hilbert space $\mathcal{H}_B$ as $$\mathrm{Tr}_B(\rho_{AB}) \equiv \sum_i(I_{A} \otimes \bra{i}_{B})\rho_{AB}(I_{A} \otimes \ket{i}_B)$$
Since $\ket{i}_B$ forms an orthnormal basis for Hilbert space $\mathcal{H}_B$, the reduced density operator of $\rho_A$ is defined as $$\rho_A = \mathrm{Tr}_B(\rho_{AB})$$
This can be thought of as "Tracing out" or "Discarding" information related to system $B$  
In pennylane this partial trace can be computed as demonstrated in the following codercises

#### Solution to Codercise N.3.2.a
Create the following entangled state $$\ket{\psi} = \cos{\left(\alpha\over{2}\right)}\ket{00} + \sin{\left(\alpha\over{2}\right)}\ket{11}$$

In [5]:
def create_entangled(alpha: float):
    """
    inputs
    ======
    alpha: is the angle through which the subcircuit is rotated
    """
    qml.RY(alpha, wires=0)
    qml.CNOT([0,1])

##### Explanation for the above
By convention all quantum wires start in the $\ket{0}$ state. Therefore for a circuit consisting of 2 Quantum Wires both will start in the state $\ket{00}$  
`qml.RY` Applied to the first wire (indexed by `0`) has the following effect $$RY\left(\alpha\right)\ket{0} = \cos{\alpha\over{2}}\ket{0} + \sin{\alpha\over{2}}\ket{1}$$ On the first wire. Therefore the resulting state for two wires that started out as $\ket{00}$ will now be $$\left(\cos{\alpha\over{2}}\ket{0} + \sin{\alpha\over{2}}\ket{1}\right) \otimes \ket{0}$$ $$= \cos{\alpha\over{2}}\ket{00} + \sin{\alpha\over{2}}\ket{10}$$
<hr>

Next we apply the `CNOT` gate with the first wire as the control and the second as the target resulting in $$\cos{\alpha\over{2}}\ket{00} + \sin{\alpha\over{2}}\ket{11}$$

#### Solution to Codercise N.3.2b
Use the above function and return the density matrix based on only the `0`th wire

In [6]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def reduced_entangled(alpha: float) -> np.array(complex):
    # Prepare the state using create_entangled
    create_entangled(alpha)
    
    return qml.density_matrix([0])

In [7]:
"""
Test for N.3.2b
"""
alpha = np.pi/3

print("For alpha = pi/3, the reduced density matrix is {}".format(reduced_entangled(alpha)))


For alpha = pi/3, the reduced density matrix is [[0.75+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j]]


When the reduced density matrix for each wire is $I\over{2}$ this is known as a maximally entangled state. One example of this is the following _**Bell State**_ $$\ket{\Phi^+}_{AB} = {{\ket{0}_A \ket{0}_B + \ket{1}_A \ket{1}_B}\over{\sqrt{2}}}$$
Another mixed state $$\rho_{AB} = \pi_{A} \otimes \pi_{B}$$ also gives equivalent reduced states, however its predictions are very different from $\ket{\phi^+}\bra{\phi^+}_{AB}$  
Using the following parity operators we can distinguish between the two $$\Pi_{even} \equiv {{1\over{2}} \left(I_{A} \otimes I_{B} + Z_{A} \otimes Z_{B} \right)}$$, and $$\Pi_{odd} \equiv {{1\over{2}} \left(I_{A} \otimes I_{B} + Z_{A} \otimes Z_{B} \right)}$$
<hr>

##### Solution to codercise 4.3.3

In [8]:
device = qml.device('default.mixed', wires = 2)

parity_even = 0.5*qml.PauliZ(wires=0) @ qml.PauliZ(wires=1)+ 0.5*qml.Identity(0) @ qml.Identity(1)
parity_odd = - 0.5*qml.PauliZ(wires=0) @ qml.PauliZ(wires=1)+ 0.5*qml.Identity(0) @ qml.Identity(1)

max_mixed = np.eye(4)/4
psi_plus = qml.math.dm_from_state_vector(np.array([1,0,0,1])/np.sqrt(2))

@qml.qnode(device)
def parity_check_circuit(state: np.array, parity_operator):
    # PREPARE THE STATE
    qml.QubitDensityMatrix(state, [0,1])

    # RETURN THE EXPECTATION VALUE OF THE PARITY OPERATOR
    return qml.expval(parity_operator)


In [9]:
"""
Test for N.3.3
"""
print("Maximally mixed state expectation values")
print(f"Odd Parity: {parity_check_circuit(max_mixed,parity_odd)}")
print(f"Even Parity: {parity_check_circuit(max_mixed,parity_even)}")

print("Maximally entangled state expectation values")
print(f"Odd Parity: {parity_check_circuit(psi_plus,parity_odd)}")
print(f"Even Parity: {parity_check_circuit(psi_plus,parity_even)}")

Maximally mixed state expectation values
Odd Parity: 0.5
Even Parity: 0.5
Maximally entangled state expectation values
Odd Parity: 0.0
Even Parity: 0.9999999999999998
